In [11]:
import pandas as pd
import numpy as np 

data = pd.read_excel('./Online Retail.xlsx',)

# Replacing data of unnecessary desc

In [91]:
replace_data = pd.read_csv('./replace1.csv')


In [18]:
data['bill_amount'] = data.UnitPrice * data.Quantity

## Removing blank, mistakes or outlier 

In [19]:
data[data.Quantity <0].shape,data[data.UnitPrice<=0].shape, data[data.CustomerID.isnull()].shape

((10624, 9), (2517, 9), (135080, 9))

In [20]:
data = data[~((data.Quantity <0)  |(data.UnitPrice<=0) | (data.CustomerID.isnull()))]

In [29]:
d = ["Bank Charges",
"CARRIAGE",
"DOTCOM POSTAGE",
"Manual",
"PADS TO MATCH ALL CUSHIONS",
"POSTAGE"]

d = [i.lower() for i in d]
data['Description'] = data['Description'].str.lower()
data = data[data['Description'].str.contains('|'.join(d))==False]
data.StockCode = data.StockCode.astype(str).str.strip()
for rep_d in replace_data.iterrows():
   data.loc[data.StockCode == rep_d[1]['id'],'Description'] = rep_d[1]['desc'].lower()

In [37]:
replace_data2 = pd.read_csv('./replace2.csv')
for rep_d in replace_data2.iterrows():
   data.loc[data.Description.str.strip() == rep_d[1]['desc'].lower().strip()
            ,'StockCode'] = rep_d[1]['id']

### Summarizing the data based on Customer for better clustering

In [50]:
data_g = data.groupby(['CustomerID']).agg(no_bills=('InvoiceNo', pd.Series.nunique), 
                                 sum_bill=('bill_amount', np.sum),
                                 no_item = ('InvoiceNo', 'count'),
                                Recency=('InvoiceDate', max),
                                 Recency_min=('InvoiceDate', min)).reset_index()

In [52]:
data_g['no_item_per_bill'] = data_g.no_item/data_g.no_bills

### Removing Outlier based on number of trips/bills

In [63]:
data_c = data_g[data_g['no_bills']<=21]
data_c.head()

,CustomerID,no_bills,sum_bill,no_item,Recency,Recency_min,no_item_per_bill
0,12346.0,1,77183.60,1,2011-01-18 10:01:00,2011-01-18 10:01:00,1.00
1,12347.0,7,4310.00,182,2011-12-07 15:52:00,2010-12-07 14:57:00,26.00
2,12348.0,4,1437.24,27,2011-09-25 13:13:00,2010-12-16 19:09:00,6.75
3,12349.0,1,1457.55,72,2011-11-21 09:51:00,2011-11-21 09:51:00,72.00
4,12350.0,1,294.40,16,2011-02-02 16:01:00,2011-02-02 16:01:00,16.00


### Normalization of data using log

In [71]:
import math
data_c['summ_bill_log'] = data_c.sum_bill.apply(lambda x: math.log(x))
data_c['Average_bill_log'] = data_c.Average_bill.apply(lambda x: math.log(x))

C:\Users\Nauman_\AppData\Local\Temp\ipykernel_19400\3579040085.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_c['summ_bill_log'] = data_c.sum_bill.apply(lambda x: math.log(x))
C:\Users\Nauman_\AppData\Local\Temp\ipykernel_19400\3579040085.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_c['Average_bill_log'] = data_c.Average_bill.apply(lambda x: math.log(x))


In [70]:
data_c['Average_bill'] = data_c.sum_bill/data_c.no_bills

C:\Users\Nauman_\AppData\Local\Temp\ipykernel_19400\12442651.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_c['Average_bill'] = data_c.sum_bill/data_c.no_bills


#### Identifying Recencency

In [74]:
data_c.loc[data_c.Recency.dt.year <2011,'Recency_int'] = 0
data_c.loc[data_c.Recency.dt.year >=2011,'Recency_int'] = data_c.Recency.dt.month

C:\Users\Nauman_\AppData\Local\Temp\ipykernel_19400\3219809033.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_c.loc[data_c.Recency.dt.year <2011,'Recency_int'] = 0


In [75]:
# identifying the duration of customer 
data_c['month_interval'] = (data_c['Recency'] - data_c['Recency_min']).dt.days/30

C:\Users\Nauman_\AppData\Local\Temp\ipykernel_19400\545425094.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_c['month_interval'] = (data_c['Recency'] - data_c['Recency_min']).dt.days/30


In [79]:
data_c['no_of_visit_per_month'] = data_c['month_interval']/data_c.no_bills

C:\Users\Nauman_\AppData\Local\Temp\ipykernel_19400\3384179801.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_c['no_of_visit_per_month'] = data_c['month_interval']/data_c.no_bills


In [84]:
data_country = data[['CustomerID','Country']].drop_duplicates()

In [92]:
# adding data Country Column on Each Customer
data_final = data_c.merge(data_country, on =['CustomerID'])

In [90]:
data_final.to_csv('final.csv')